In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Tokenizing text

In [1]:
import re
text = "Hello, world. This, is a test."
result = re.split(r"(\s)", text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


split by punctuation as well

In [3]:
result = re.split(r"([,.]|\s)", text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


remove whitespaces

In [4]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


handling all types of special characters

In [7]:
text = "Hello, world. Is this-- a test?"
result = re.split(r"([,.:;?_!\"()\']|--|\s)", text)
result = [ item.strip() for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']

Apply this tokenization on the text of interest

In [9]:
preprocessed = re.split(r"([,;.:?_!\"()']|--|\s)", raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [10]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# Converting tokens into token IDs

Building vocabulary for token to integer mapping

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [14]:
vocab = {token: integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab):
    print(item)
    if i >= 10: break

!
"
'
(
)
,
--
.
:
;
?


*Apply vocabulary to convert new text into token ids*
**Tokenizer Class**

In [18]:
class SimpleTokenizerV1:
    """
    Tokenizer class with an encode method that splits text into tokens and carries out the string-to-integer
    mapping to produce toke IDs via the vocabulary
    """
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # remove spaces before specified punctuation
        return text

In [24]:
tokenizer = SimpleTokenizerV1(vocab=vocab)
text = """"it's the last he painted, you know,"
        Mrs. Gisburn said with pardonable pride"""

ids = tokenizer.encode(text)
print(ids)

[1, 585, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793]


In [25]:
print(tokenizer.decode(ids))

" it' s the last he painted, you know," Mrs. Gisburn said with pardonable pride


In [26]:
# Apply encoding to new text sample not contained in the training test
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

# Adding special context tokens

We need to modify the tokenizer to handle **unknown words**. We also need to address the usage and addition of **special context tokens** that can enhance a model's understanding of context or other relevant information in the text

In [31]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

print(len(vocab))

1132


In [35]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [36]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_text = { i: s for s, i in vocab.items() }
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int(item) for item in preprocessed]
        return ids

    def decode(self, ids):  

        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.;:?!"()\'])', r'\1', text) # remove spaces before specified punctuation
        return text      